In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor_research.bancor_simulator.v3.spec.network import BancorNetwork

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorNetwork(whitelisted_tokens=whitelisted_tokens)

# The actual deposits.

v3.deposit(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.deposit(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens, 'bnt')

v3.trade(tkn_amt=2000, source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)

v3.trade(tkn_amt=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)

v3.trade(tkn_amt=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=0)

def read_url(url):
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# TKN Surplus Example
A TKN is in surplus if the total vault balance is greater than the staked amount, after accounting for the exit fee. It is necessary to separate the vault balance into the trading liquidity (b), and non-trading liquidity (c) components; b + c is the vault balance of TKN:

![](https://lh3.googleusercontent.com/TaB48nvgG3iMgdutkQdjP1KVXNQlabsQ2DWDdBcdau19YjsFh0U6L12_-fSJ23KrYDRNxPZQE3sYGc5PGeb-PT1FgzrsvWgweFaB6wI88SGlRaWaVAzC_3d8OJW5oyP2LoOtaNdy)

where e is the TKN balance of the staking ledger, and n is the withdrawal fee (e.g. 0.0025, or 0.25%). So long as the above expression evaluates as True, the protocol is in an effective surplus (i.e. the quantity of TKN exceeds that of the combined user stakes). Therefore in all cases, it is guaranteed that any user withdrawing will receive only TKN. However, the protocol will also attempt to reduce the surplus and recover BNT from the secondary markets. The following paragraphs will explore the three different situations that cover the discrete behaviors of the withdrawal algorithm during a surplus state of TKN.



The most important decision the protocol makes is to evaluate the abuse vector described above. To do this, two important thresholds are calculated, hlim and hmax; the former determines whether there is sufficient non-trading TKN in the vault to support the user's withdrawal without changing the trading pool depth, and the latter determines if an apparent abuse vector exists. These two values are calculated as follows:

![](https://lh3.googleusercontent.com/yCN3qagE1MLNA24RqemiPLcPOD1_9fcR2wW0ZnlY2I_sBd_pebcdEy3veURQTeeV8HFk4iIKGkqyKILvHb8peoelQNN7ddnXBWHOtWFSyrkUTC2OXaJx9vwziVRKIju_dGilKM9t)

![](https://lh6.googleusercontent.com/Fe3pLh_WmrFLO8F2KjltvdrA3fpu1Z4god1adfFeRl_dhBD0aSCOLV9evkd9GC7SNYWbpDQKxd6Yp-zuzeyuFJoPTmdqwnLGnr4-Zlgb_oWgSAJwOHxRprytjamBo6zXAbldCQ82)

These two values, hlim and hmax, define the maximum value for x that is allowable without creating a possible abuse incentive. Therefore, the user's withdrawal amount (x) must be less than both hlim and hmax:

![](https://lh3.googleusercontent.com/F9tBEhoDYsFtiDmCwTKoKjesPyCtxswCVOrZp7US5cwbsHHoVlQ6g2A35RckcliVov-u34yXsT_h7dvsGwzeYnFbV1vVt5xao81akBT4m2OmqwMkXMIerV52ED-VQaWwIfKxpJR1)

![](https://lh4.googleusercontent.com/vBVgt4G_ZfiJGBE7mp7DR1pKP017Qm9idEsFCkPNolOG_uRQ_LgQlZCCp9lUpfx2fEgzED3ir218PJG69163oXXesIQwC7LzSSTeEWa8vMTEexd9z0hsTsJpFeRB8Va67gtFSp24)

To examine this further, consider the following case: the staking ledger reports a staked balance of 1,400 TKN (e), the vault balance of TKN is 1,500 TKN (b+c), the TKN trading pool has 1,000 TKN (b) and 1,000 BNT (a) in available liquidity. Then, a user confirms a withdrawal for 100 TKN (x).

![](https://lh4.googleusercontent.com/IFZqqWddyvXZFCv0KYqOzZnnim6U5_XUZmoafI4MbBATCeZrlwXsKE6_pN3dQ1bA_l57fNSvFqc8wtc5YGtRkijraysa_NbcNZ5X3GR4BVmb6WWX7YCK5bdZlO4LBujr-RI41UWz)


In this case, hlim evaluates to 466.6667 TKN, and hmax evaluates to 501.486064 TKN. As the user's withdrawal amount (x) is less than both of these values, there must be sufficient non-trading TKN in the vault to support the withdrawal, and there is no financial incentive to create the withdrawal for the purpose of back running. Therefore the protocol can attempt to recover a small amount of BNT from the outside markets.

The withdrawal algorithm has six different outputs:

P, BNT quantity to remove (burn) from the available trading liquidity.

Q, BNT ownership renounced by the protocol (i.e. removed from the staking ledger).

R, TKN quantity to add to the available trading liquidity from the non-trading liquidity.

S, TKN quantity to remove from the vault (and send to the user).

T, BNT quantity minted for, and sent to the user as compensation.

U, TKN removed from the external protection wallet and sent to the user as compensation.

The method for calculating each of these outputs is dependent on the state of the system. The case where there is a TKN surplus, and the user's withdrawal satisfies the hlim and hmax checks, the outputs are calculated as follows:

![](https://lh4.googleusercontent.com/IGcP7c7JT-4O2SHxryivqMZlK-g0htlQUtA9PF-0xdU1werfURoFdCj3crbf4868XuvR_j5fGtgnAabC9zy6Vd0AHlIJc23lsfmwJE5pxitZIyPFyF0NT7lSVKIis_aCtH6b7-jE)

![](https://lh5.googleusercontent.com/1dJR9I0xUgc69g-zzrGb7Bn_Z8u3e8RpmuAqsr75v_QeKhnxPE9ZTX7_QD9tUglgQwn9Clyi3efutsVDyF__X1-9DE8oi52KMuNl0fC3r4KTU_oh4mIoPJ5uu9e2kLtpAQ60FrzG)

![](https://lh6.googleusercontent.com/jhqsmvPF6jpGs1RVPFwUBuWQoAZR0x07J2z5UayF45uEWDjBDkoI94-Ac65bKNSgDQp5tr3qgv4UCVUXdt8P8L9rk6veA_ESnEdE4YlAUazLmJhD1S-KNYi1aKZ7fzWpOfZUPm6-)

![](https://lh6.googleusercontent.com/fyEcUmUKV6HbK0KBC67nlMYecAcs_1bYFn9abnsWi2GjL9ZklTiQAPvFBeafVo6RIVuh6HuJUbKs4gqOfassQJDbKNc6Kwg2BfHgVEpOICJlwyHCCWf6HsPDaD6j2sBMxgt0la-m)

![](https://lh3.googleusercontent.com/LayQKPlGXjqb-6jq69T8bl2u6mWpHBqOI50lYGkIvltdnWAXWBgb25y1zXETAye0_sGen-Jeod0LXgdrvQkzdqoT5PW4qXuaDzodNJ1UZV6KEdBZomnXCpW57lgS235-YD1QNO6K)

![](https://lh5.googleusercontent.com/ZdiyH2x66LamFTxwgDQSa-jo1wVteZnJjtVfwSulgUmVJamCch-JYL8_8966qVoJ8Od0Lbz8v8YyiuULu87zfO9h7NpIci-eJuhEeGBpHV39MqeYnBJh6vupPnR0k007hrat6IPY)

![](https://lh6.googleusercontent.com/OTjBdYrof9Bbw-gBcafwaEjb-_7GFiW0D7G3b8CQltplGTWVm26Xo6oXrIiPfPt-IlUxyRQlC-2vNSASWjKOzRfBEXhOdroaI6wGtcVL6mPxaRx-SF6sTQ9FNFJ5dQCaYY4JKhGx)

![](https://lh6.googleusercontent.com/bhvNFRn2wsVFqlR6ZcfCn5GKsy-eQRyXlXSaBi24FtzEMXGz_RvG5B5tazU_sUyHX9BgJnaERuDExgRLyYSs12wlm25KStMguoriZvysMmOhTB-8yzD__1WR2PjKzzMDFanN67Ck)

It is important to note that during a surplus, the algorithm reacts by moving the price of BNT vs TKN in the pool; note that the pool began with a spot rate of 1:1, and arrived at a slightly higher price for BNT at the conclusion of the process. This repricing is such that, if and when the pool returns to balance (defined as state of the pool before the withdrawal was executed), the relative surplus of TKN compared to the staked balance will remain constant, (ignoring vortex fee effects). In essence, either the valuation of BNT is allowed to increase vs TKN, or the closing of a future arbitrage opportunity positively affects the total BNT balance of the protocol. The number of BNT burned from the trading liquidity is equal to the amount that would be returned in a hypothetically perfect arbitrage, assuming the rate inside the pool is in agreement with consumer expectations (i.e. the rest of the market). This anticipatory BNT is part of the process - while BNT is lost from the trading liquidity in this case, the protocol does not relinquish its pool tokens. It might be helpful to think of this as a small trade of TKN to BNT having just occurred. Whether or not the apparent arbitrage opportunity is closed immediately is irrelevant; for example, it is likely that these small adjustments will occasionally close existing arbitrage opportunities. It is only important that a compensatory action is taken by the protocol that addresses the implicit BNT valuation and/or its liquidity equilibrium state.

Such actions can only be taken if the hlim and hmax tests evaluate to True. To examine a case wherein these tests do not pass, consider a scenario nearly identical to that presented above, but where the non-trading TKN balance (c) is increased to 1,000 TKN. Therefore, the staking ledger reports a staked balance of 1,400 TKN (e), the vault balance of TKN is 2,000 TKN (b+c), the TKN trading pool has 1,000 TKN (b) and 1,000 BNT (a) in available liquidity, and the user confirms a withdrawal for 100 TKN (x).

![](https://lh5.googleusercontent.com/ljTvQHEnZe-h5zYr7qLyL0B0mx7YgzNgBQbA3Wkl1ISVmlSPZxtr68lyWUo6DY-wGkAzqYKbqu0dozgtVFcDymJ7OmCaKjBZ4FARzHeX-H1-z29InlKuxPfllW3Vt-eClg_pty4g)

In this case, hlim evaluates to 700 TKN, and hmax evaluates to 18.208192 TKN. For this scenario, the hlim test still passes, but the hmax test fails. Therefore, the protocol's repricing strategy is potentially abusable, and the withdrawal algorithm will default to a different set of operations. As the protocol prefers to not interfere with the depth of the pool where possible, a bifurcation in the logic at this point is required. In essence, the protocol asks if there is sufficient TKN in the non-trading vault balance to cover the user's withdrawal.

![](https://lh6.googleusercontent.com/NiMtFsqayF4KDfhzheLuV4XLkf9wW2OQBu26kNvh9Kyb85jO1ASjT0IqAfbcViFOpcCZWmxbXErmizdc_tV_Du_j50emJttgQiOJSwWcn0jOKm1AVLSYFklcJIFbGszxmDI59cl9)

In this case the inequality is True, which essentially means that the user's withdrawal can be processed without touching the trading liquidity. A modified method for calculating each of the algorithm outputs is necessary:

![](https://lh4.googleusercontent.com/lJf6kRUn2_IW9grxaUiUMQo8suVq1erzjWOGY8o6huBTp_vXCUpUvSt415N8FYnj15zXdStYy1YYZ3_RZVGJvkfB_wj1Z_tRsdOczABQj7cnE2X3o3WUwm1r6I5L-e2HAxF1LNy4)

![](https://lh5.googleusercontent.com/4mufywW-XWRfF8aVDjZr2Jrr1sMWmBTGrgac2VFob4P-ZVPeIzkCFAkKbP7aS_7M_tvnXmb0zpBM900vCovVh6eO1WzBvQhdvXGNXCL6WGCXT9_25A3ykGZX22vzNfIJ51R5dbti)

![](https://lh4.googleusercontent.com/X6sBpEapvzU3MYooLiiqY4oDwUuTFoet8sQi8UinQGYhZ_zcwPg5Qef1ZgBQuEp0w8lM7HLmvori_sP6o6M2G4nkzz32dYm7AJATMjE_n4ArM_Of9kD59NCqIHl04qXSkHuOV5HE)

![](https://lh5.googleusercontent.com/P10bCvI_TvmcqGxSvURUzwmw-VID0dAlrfBZCAjm9Zdj4fVWVAAuGeFcntKyIgTm2g8Mg8MOXGwCzjh6PPV7a250xOH9Kg1VpzH74wtvKXJEp8401v0lMlV1SZt4OQHLpkvg3RFx)

![](https://lh3.googleusercontent.com/s1CebBQRLMzMDbLOj_1B75RHcEzMivNjY3Wlu4Qs5I1J38mP8AAxBqqMPtwdqLIUXL_qVjSrJp7iMP0oET9uOBPESaFQDhcNr_MvYz-bEEUQIRgQrFC_UDs4SXrviB0emgZLyJ9v)

![](https://lh4.googleusercontent.com/09bKRp73aJU-WjjAoeO8HVtoCCAMjqGluaU4k3aOh2XmpZ64sOAHxK-PsdV_IZFMZTB8eH4_zZrMaGD2Mdy9g5VVzUac1446VKYET6OOn0xuNkGXro4YO-IlGxLDYMIQ_ZtvNGen)

![](https://lh6.googleusercontent.com/qPosWRZZL8JeyqsGiVsxjwfKzkFGleIKwp3Dz-E9Anv7PFZyhJcwE7tctFw_TFlH9jouaS2c0T6R0GowHyFASDrGxFQxOfXAR1Pi33Weuju9AIxNNPwI6KwaQqsmLiZ0IFgJckOJ)

![](https://lh3.googleusercontent.com/XhG-d5Qm0aZ49Q9b6GGy_0jdbpvv2JeZIo2XPE5XL7edWNymNLz5gPKjG-PzumGcj5kv2old7T1pumLGLKvo7RKfKP3y9gRkRCqzNPt_FUD7f1Noux07VNv6VbryoW8TdF_tCSli)

This is the simplest case to understand. The user withdraws their TKN directly from the vault, and no compensatory actions are taken. The last remaining case for withdrawals is when there are insufficient non-trading funds to process the user's withdrawal - when the inequality presented above is False. Consider the situation where the vault balance of TKN is the same, but the proportion of funds used for trading liquidity is much higher. Instead of a pool only 1,000 TKN deep, let it be 1,995 TKN deep instead, while maintaining the same vault balance (i.e. the non-trading TKN balance (c) is reduced while the trading liquidity balance (a) is increased). As before, the staking ledger reports a staked balance of 1,400 TKN (e), the vault balance of TKN is 2,000 TKN (b+c), the TKN trading pool has 1,995 TKN (b) and 1,995 BNT (a) in available liquidity, and the user confirms a withdrawal for 100 TKN (x).

![](https://lh5.googleusercontent.com/BpFF_JGwwUBTg1t9SRlrknNrVXysm9GFnEDc8R0kEldwRlDUSgUV9FRQgku9-s8qkUp8qHMp74XZqRHqa0XF7ZWV9h0OsiaBEyZkT7zMLIlJCpfHAtqZ4jtJW1iGOQnxQ8fUJJny) 

In this case, hlim evaluates to 3.5 TKN, and hmax evaluates to 36.325343 TKN. For this scenario, both of the hlim and hmax tests fail. As before, the protocol detects that its repricing method can be abused, and proceeds to check if the available non-trading liquidity is sufficient to cover the withdrawal.

![](https://lh6.googleusercontent.com/NiMtFsqayF4KDfhzheLuV4XLkf9wW2OQBu26kNvh9Kyb85jO1ASjT0IqAfbcViFOpcCZWmxbXErmizdc_tV_Du_j50emJttgQiOJSwWcn0jOKm1AVLSYFklcJIFbGszxmDI59cl9)

In this case the inequality is False. Therefore, it is necessary to reduce the available trading liquidity. A third series of methods is required to calculate each of the algorithm outputs:

![](https://lh5.googleusercontent.com/auG6JYDh3hPLvVbQiGPR4rHWQI3JwOKGnHIrRHKUz8iXf4DIn_0tCtbfM03TXPDX31rr4ZRhd87kR9KipB82OmYaAjZnrnDTtCMdaMWXiO7afWMphoDZj24ROLwMM5Qy4sEro970)

![](https://lh3.googleusercontent.com/8NP-0fdnQwYGAb72Ry96evZtKJr-92bUKdyAsSo-zm8KFPlqyrUNytILov0ybvvzzJ-TqH0IhFhEZfs3uEeAfCaE_v5HZ28tjFS11jo_cYGFRH37TSxphJPkxqAyiyk2GSsjli7V)

![](https://lh6.googleusercontent.com/N9-oshT9IJe8adi77_sgP95S5evuH4EV28A2FaKKFh0RigoxuxItVdbldna0TNo9lZ-o6Lr6rZ2BbGVswwrgbz5vNoeDkhcTdBTBsuB2wfAmWOyLfH8HNppDH5_5UKA5AAQ1MqeZ)

![](https://lh4.googleusercontent.com/HSw2eNMD9po8I_eASHazKbF1LWzt85i-zukvz110PrBSyntox2Wt-iQw39tdD3xOny2W9uMsvJslxDNKQWWgBGizCXrfp3Dn2eGjw6jnra1ewN00PcyhEy5sIT55QF2QFrhJ2uhZ)

![](https://lh5.googleusercontent.com/fgV6-iQviBEazgHzfairDTFUSu6vjyJ0-eNdT35jyWY3w1HoV9Jop2cXbeqJ9cLK5W-YOWMwKonVNGBY9QMKMYrf1ac9cehskQl7y31cAeiFzN-_-N3myoNKfQjrtIR5siDSQWEW)

![](https://lh4.googleusercontent.com/LqsJ4i34dyolY71eYTReCQYK_m9wnGhjNIXVIXF4Ltz9mTfT_3e7d5NitXdeIKiEHG-JCTss25JlOsJlIYtngHKAUP9otkXbO3GGvMX1e20mhms9Y2qA-u11VmLkcAbT09YsE9nG)

![](https://lh4.googleusercontent.com/ixI5rgHtGFfzuMOegeYt2r1AozbdzgxWFqS9XuBSGV5TB2F2HFd4LbVaUZUx1RcN7r43ylYk5-XQaJul976-nDOTns34IiLmHarSMOsmMyRCJUUnYszvpi3AVVbctBtEM5FQG6UI)

![](https://lh5.googleusercontent.com/ewJlMMPJZPAAUQlRYraYoFTp14AkkzrtmNAOXBIseIUvlbVx2Hf95MCyA5hRErJvJIMbCOnIfWaiTTkq9TaMQhg2BHbF8Wuam-MdPJf51GTEJnTPIcBLTqzcALGwJgaX86lyzx7s)\
To summarise the outcome for this scenario, the protocol gives up what is left of the non-trading TKN, and then the remaining balance is taken from the trading liquidity. Since the pool depth is directly affected, the protocol must renounce the BNT component of the pool. Previously, the variables Q and R were treated independently, as the small change in the BNT trading liquidity was akin to a simulated swap, rather than a formal withdrawal. However, in this case Q = R, as the reduction in the pool is appropriately treated as the protocol performing a withdrawal. The Q quantity is denominated in BNT, and is later processed into its equivalence in bnBNT; both the BNT and the bnBNT are destroyed, and the staked balance of BNT is adjusted down, accordingly.
Let us do this in the context of a simple example, and then use the method with a larger dataset.